### NAVER

In [ ]:
import requests
import sqlite3
import re
import pickle

#### 불용어 제거

In [ ]:
def check_title_delete_article(title):   
    # 기본 정규식
    reg_ex = {'email': r'[a-zA-Z0-9.!#$%&\'*+/=?^_`{|}~-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+',
              'sb': r'\[[^\]]*\]',
              'pb': r'\<[^\>]*\>'}
    # 매칭되면 기사를 삭제하는 태그, 단어 및 정규식
    stop_title_sb = ['[포토]', '[프로필]', '[★화보]', '[게시판]', '[경향포토]',
                     '[리빙포인트]', '[머니S포토]', '[부고]', '[부음]', '[사람 人 사람]',
                     '[사진]', '[서울포토]', '[영어]', "[오래전 '이날']", '[오마이포토]',
                     '[이 시각 코스피]', '[이 시각 코스닥]', '[인사]', '[코스닥 공시]', '[코스닥(개장)]',
                     '[코스닥(마감)]', '[코스피(개장)]', '[코스피(마감)]', '[포토뉴스]', '[표]',
                     '[한경로보뉴스]', '[영상]', '[오늘의 국회 ', '[오늘의 주요일정]']

    stop_title_pb = ['<부고>', '<오늘의 조간 정치뉴스>', '<인사>', '<포토>', '<코>', '<유>']
    # Square brackets tag로 시작할 경우
    p = r'^' + reg_ex['sb']
    m = re.match(p, title)
    # 매칭 안 되면 다음 정규식 검사로 넘어감.
    if m == None:
        pass
    # 매칭된 문자열이 불용어 목록에 존재하면 True 반환하여 해당 기사 삭제
    elif m[0] in stop_title_sb:
        return True
    
    # Pointy brackets tag로 시작할 경우
    p = r'^' + reg_ex['pb']
    m = re.match(p, title)
    if m == None:
        pass
    elif m[0] in stop_title_pb:
        return True
    
    # 매칭 되지 않거나 매칭된 문자열이 불용어 목록에 존재하지 않으면 False 반환하여 해당 기사 수집
    return 

In [ ]:
def check_content_delete_article(content):
    stop_content_reg_ex = [r'^동영상[\s]*뉴스']
    # 특정 단어로 시작할 경우
    for p in stop_content_reg_ex:
        m = re.match(p, content)
        # 매칭 안 되면 다음 정규식 검사로 넘어감.
        if m == None:
            pass
        # 매칭될 경우 True 반환하여 해당 기사 삭제
        else:
            return True
    # 매칭 되지 않거나 매칭된 문자열이 불용어 목록에 존재하지 않으면 False 반환하여 해당 기사 수집
    return False

In [ ]:
def check_content_crop_content(content):
    # 기본 정규식
    reg_ex = {'email': r'[a-zA-Z0-9.!#$%&\'*+/=?^_`{|}~-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+',
              'sb': r'\[[^\]]*\]',
              'pb': r'\<[^\>]*\>'}
    stop_content_reg_ex_crop = [reg_ex['email']]

    stop_content_reg_ex_ml_crop = [r'^[\s]*\【.*?\】([^=\n]*기자[\s]*=)?',
                                   r'^[\s]*\[.*?\]([^=\n)]*기자[\s]*=)?',
                                   r'^[\s]*\(.*?\)([^=\n]*기자[\s]*=)?',
                                   r'^[ㄱ-ㅎㅏ-ㅣ가-힣]+[\s]?기자[\s]?',
                                   r'^© News1.*\n',
                                   '(- Copyrights )?ⓒ[^\.]*$',
                                   r'\【.*?\】[\s]*$',
                                   r'\[.*?\][\s]*$',
                                   r'\(.*?\)[\s]*$',
                                   r'[\s\.][ㄱ-ㅎㅏ-ㅣ가-힣]+[\s]?기자$']
    # Normal mode
    for p in stop_content_reg_ex_crop:
        # 매칭될 경우 매칭된 문자열 잘래낸 후 앞뒤 공백 제거
        content = re.sub(p, '', content).strip()
    
    # Multiline mode
    for p in stop_content_reg_ex_ml_crop:
        content = re.sub(p, '', content, flags=re.MULTILINE).strip()

    # 잘라낸 content 반환
    return content

#### 해당 링크에서 뷰티풀솝으로 파싱

In [ ]:
def parsingArticleReq(link, headers, sect):
    try:
        article = requests.get(link, headers=headers)
    except:
        article = requests.get(link, headers=headers, verify=False)
    article = BeautifulSoup(article.text, "lxml")
    url = link
    a_id = link.split('aid=')
    a_id = "na_"+a_id[1]
    try:
        articleTitle = article.select_one("#articleTitle")
    except:
        return
    else:
        articleTitle = articleTitle.text.strip()
        if check_title_delete_article(articleTitle):
            return
        dateTime = article.select(".t11")
        dateTime = dateTime[0].text.split(" ")
        date = dateTime[0]
        time = dateTime[1]
        reg_ex = {'email':r'[a-zA-Z0-9.!#$%&\'*+/=?^_`{|}~-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+'}
        cont = article.select_one("#articleBodyContents").text
        if check_content_delete_article(cont):
            return
        cont = check_content_crop_content(cont)
        # content : 기자 이메일 직전 개행문자부터 이하 내용 제거
        try:
            p = re.findall(r'.{20}' + reg_ex['email'], content, flags=re.DOTALL)[-1]
            email_idx = re.search(p, cont).start() + 20
            content_temp = cont[:email_idx].strip()
            # 이메일이 본문 앞이나 중간에 들어가지 않은 경우에만 잘린 본문 사용
            if (len(content_temp) / len(cont)) > 0.6:
                cont = content_temp
        except:
             pass
        cont = cont.replace("// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}","").strip()
        if cont.find('▶') != -1:
            tempIdx = cont.index('▶')
            if tempIdx > 0:
                cont = cont[:tempIdx]
        res = re.search(r"\w+\s+기자",cont)
        if res != None:
            author = res.group()
            author = author.replace(' 기자','')
        else:
            author = str()
        press = article.select_one(".press_logo a img")
        press = press.get('title')
        section = sect
        data = (a_id, date, time, articleTitle, cont, press, author, section, url)
        try:
            cur.execute("INSERT INTO Article(a_id, date, time, title, content, press, author, section, url) \
                        values(?,?,?,?,?,?,?,?,?)", data)
        except:
            data = ''
        else:
            conn.commit()

#### 다른포맷

In [ ]:
def diffFormatCrawling(link, headers, sect):
    try:
        article = requests.get(link, headers=headers)
    except:
        article = requests.get(link, headers=headers, verify=False)
    article = BeautifulSoup(article.text, "lxml")
    url = link
    a_id = link.split('aid=')
    a_id = "na_"+a_id[1]
    articleTitle = article.select_one('title').text.strip()
    if check_title_delete_article(articleTitle):
            return
    dateTime = article.select('.info span')[1].text
    dateTime = dateTime.split(" ")
    date = dateTime[1]
    time = dateTime[2]+" "+dateTime[3]
    reg_ex = {'email':r'[a-zA-Z0-9.!#$%&\'*+/=?^_`{|}~-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+'}
    cont = article.select_one("#newsEndContents").text
    if check_content_delete_article(cont):
            return
    # content : 기자 이메일 직전 개행문자부터 이하 내용 제거
    try:
        p = re.findall(r'.{20}' + reg_ex['email'], content, flags=re.DOTALL)[-1]
        email_idx = re.search(p, cont).start() + 20
        content_temp = cont[:email_idx].strip()
        # 이메일이 본문 앞이나 중간에 들어가지 않은 경우에만 잘린 본문 사용
        if (len(content_temp) / len(cont)) > 0.6:
            cont = content_temp
    except:
        pass
    cont = cont.replace("// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}","").strip()
    if cont.find('▶') != -1:
            tempIdx = cont.index('▶')
            if tempIdx > 0:
                cont = cont[:tempIdx]
    res = re.search(r"\w+\s+기자",cont)
    if res != None:
        author = res.group()
        author = author.replace(' 기자','')
    else:
        author = str()
    press = article.select_one(".link img")
    press = press.get('alt')
    section = sect
    data = (a_id, date, time, articleTitle, cont, press, author, section, url)
    try:
        cur.execute("INSERT INTO Article(a_id, date, time, title, content, press, author, section, url) \
                        values(?,?,?,?,?,?,?,?,?)", data)
    except:
        pass
    else:
        conn.commit()

#### 긁어온 링크에서 파싱

In [ ]:
headers = {
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36",
}

In [ ]:
e_date = 20180823
period = 1
path = 'db\\urls\\naver\\'
sections = ['Society']
# sections = ['politics', 'economic', 'society', 'global', 'culture', 'digital']

In [ ]:
cdates = [str(i) for i in range(e_date-period+1, e_date+1)]
for cdate in cdates:
    for section in sections:
        with open(f'{path}naver_{section}_{cdate}.pkl', 'rb') as f:
            urls = pickle.load(f)
            print(f'{path}naver_{section}_{cdate}.pkl')
            for i, url in enumerate(urls):
                if (i % 1000) == 0:
                    print(f'{i:6,}/{len(urls):6,}')
                print(url)

In [ ]:
for p in politicsLink:
    try:
        parsingArticleReq(p, headers, '정치')
    except:
        try:
            diffFormatCrawling(p, headers, '정치')
        except:
            print(p)

In [ ]:
for e in economicLink:
    try:
        parsingArticleReq(e, headers, '경제')
    except:
        try:
            diffFormatCrawling(e, headers, '경제')
        except:
            print(e)

In [ ]:
for s in societyLink:
    try:
        parsingArticleReq(s, headers, '사회')
    except:
        try:
            diffFormatCrawling(s, headers, '사회')
        except:
            print(s)

In [ ]:
for c in cultureLink:
    try:
        parsingArticleReq(c, headers, '문화')
    except:
        try:
            diffFormatCrawling(c, headers, '문화')
        except:
            print(c)

In [ ]:
for d in digitalLink:
    try:
        parsingArticleReq(d, headers, 'IT')
    except:
        try:
            diffFormatCrawling(d, headers, 'IT')
        except:
            print(d)    

In [ ]:
for g in globalLink:
    try:
        parsingArticleReq(g, headers, '세계')
    except:
        try:
            diffFormatCrawling(g, headers, '세계')
        except:
            print(g)

#### DB만들기

In [ ]:
conn = sqlite3.connect('\\db\\news_db.db')
cur = conn.cursor()

In [ ]:
try:
    cur.execute("CREATE TABLE Article(a_id TEXT primary key, \
                 date DATE, time TIME, title TEXT, content TEXT, press TEXT,author TEXT, section TEXT, url TEXT)")
    conn.commit()
except:
    pass

In [ ]:
#cur.execute("DELETE FROM naver")

#### 링크 모으기

In [ ]:
# from bs4 import BeautifulSoup
# from selenium import webdriver

In [ ]:
# driver = webdriver.Chrome()

# def collectLinks(url):
#     driver.get(url)
#     curPage = 0
#     links = []
#     pages = driver.find_elements_by_css_selector('.paging a')
#     pages = [page.get_attribute('href') for page in pages] #2부터
#     while curPage < len(pages):
#         obj = driver.find_elements_by_css_selector(".type06_headline dt a")        
#         tempLinks = [link.get_attribute('href') for link in obj]
#         links.extend(tempLinks)
#         driver.get(pages[curPage])
#         curPage += 1
#         if curPage==10:
#             curPage = 0
#             pages = driver.find_elements_by_css_selector('.paging a')
#             pages = [page.get_attribute('href') for page in pages]
#             pages = pages[1:]
#     return links

#### 특정 날짜 링크긁기

In [ ]:
# cdates = ['20180823']

# url = 'https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm'
# # sid = {'politics':'100','economic':'101','society':'102','culture':'103',
# #        'globalList':'104','digital':'105'}
# sid = {'globalList':'104'}
# politics = ['264','265','266','267','268','269']
# economic = ['258','259','260','261','262','263','310','771']
# society = ['249','250','251','252','254','255','256','257','276','59b']
# globalList = ['231','232','233','234','322']
# culture = ['237','238','239','240','241','242','243','244','245','248','376']
# digital = ['226','227','228','229','230','283','731','732']

# politicsLink = []
# economicLink = []
# societyLink = []
# globalLink = []
# cultureLink = []
# digitalLink = []

# for cdate in cdates:
#     for sid_k,sid_v in sid.items():
# #         if sid_k == 'politics':
# #             for t in politics:
# #                 temp = collectLinks(url+"&sid1="+sid_v+"&sid2="+t+"&date="+cdate)
# #                 politicsLink.extend(temp)
# #         elif sid_k == 'economic':
# #             for t in economic:
# #                 temp = collectLinks(url+"&sid1="+sid_v+"&sid2="+t+"&date="+cdate)
# #                 economicLink.extend(temp)
# #         elif sid_k == 'culture':
# #             for t in culture:
# #                 temp = collectLinks(url+"&sid1="+sid_v+"&sid2="+t+"&date="+cdate)
# #                 cultureLink.extend(temp)
# #         elif sid_k == 'society':
# #             for t in society:
# #                 temp = collectLinks(url+"&sid1="+sid_v+"&sid2="+t+"&date="+cdate)
# #                 societyLink.extend(temp)
#         if sid_k == 'globalList':
#             for t in globalList:
#                 temp = collectLinks(url+"&sid1="+sid_v+"&sid2="+t+"&date="+cdate)
#                 globalLink.extend(temp)
# #         else:
# #             for t in digital:
# #                 temp = collectLinks(url+"&sid1="+sid_v+"&sid2="+t+"&date="+cdate)
# #                 digitalLink.extend(temp)

#### 피클 저장하기

In [ ]:
# def savePickle(date):
# #     with open('db\\urls\\naver\\naver_politics_' + date + '.pkl', 'wb') as f:
# #         pickle.dump(politicsLink,f)
# #     with open('db\\urls\\naver\\naver_economic_' + date + '.pkl', 'wb') as f:
# #         pickle.dump(economicLink,f)
# #     with open('db\\urls\\naver\\naver_society_'+ date + '.pkl', 'wb') as f:
# #         pickle.dump(societyLink,f)
#     with open('db\\urls\\naver\\naver_global_' + date + '.pkl', 'wb') as f:
#         pickle.dump(globalLink,f)
# #     with open('db\\urls\\naver\\naver_culture_'+ date + '.pkl', 'wb') as f:
# #         pickle.dump(cultureLink,f)
# #     with open('db\\urls\\naver\\naver_digital_'+ date + '.pkl', 'wb') as f:
# #         pickle.dump(digitalLink,f)

In [ ]:
# savePickle('20180823')